### Imports

In [4]:
! pip install pandas
! pip install torch
! pip install numpy

import pandas as pd
import numpy as np
import torch
import csv


You should consider upgrading via the 'c:\users\vincent\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\vincent\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\vincent\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [5]:
encoding_defn = ['FATHMM_T', 'FATHMM_D', 
                 'MT_A', 'MT_D','MT_N', 'MT_P', 
                 'SIFT_T', 'SIFT_D', 
                 'LRT_D', 'LRT_N', 'LRT_U',
                 'CV_B', 'CV_LB', 'CV_LP', 'CV_P', 'CV_DR', 'CV_H', 'CV_US', 'CV_CIP']

encoding_dict = {
    'FATHMM_pred': {
        'T': 0,
        'D': 1
    },
    'MutationTaster_pred': {
        'A': 2,
        'D': 3,
        'N': 4,
        'P': 5
    },
    'SIFT_pred': {
        'T': 6,
        'D': 7
    },
    'LRT_pred': {
        'D': 8,
        'N': 9,
        'U': 10,
    },
    'clinvar_clnsig': {
        'Benign': 11,
        'Likely_benign': 12,
        'Likely_pathogenic': 13,
        'Pathogenic': 14,
        'drug_response': 15,
        'histocompatibility': 16,
        'Uncertain_significance': 17,
        'Conflicting_interpretations_of_pathogenicity': 18
    }
}


In [6]:
# Make sure path is correct
df = pd.read_csv('C:/Projects/ClinBall/out1.tsv', sep='\t')

encoded = np.zeros((len(df), len(encoding_defn)))
print(encoded.shape)
with open('C:/Projects/ClinBall/out1.tsv', 'r') as f:
    reader = csv.DictReader(f, delimiter='\t')
    for i, row in enumerate(reader):
        for key, val in row.items():
            if val == '.':
                continue
            # Fixme: Restrict possible clinvar values
            if key == 'clinvar_clnsig' and val not in encoding_dict['clinvar_clnsig'].keys():
                continue            
            j = encoding_dict[key][val]
            encoded[i][j] = 1
print(encoded[0])

torch_tensor = torch.tensor(encoded)

# # printing out result
print(torch_tensor)

(16936, 19)
[0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)


In [2]:
!pip install numpy 
!pip install pandas
!pip install torch
!pip install sklearn

import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F
from sklearn.metrics import accuracy_score

You should consider upgrading via the 'c:\users\vincent\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\vincent\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\vincent\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\vincent\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [46]:
df = pd.read_csv('C:/Projects/ClinBall/out1.tsv', sep='\t')

mapping = {
    'SIFT_pred': {
        'T': 0, 
        'D': 1,
    },
    'LRT_pred': {
        'N': 0,
        'D': 1,
        'U': 2,
    },
    'MutationTaster_pred': {
        'N': 0,
        'D': 1,
        'A': 2,
        'P': 3,
    },
    'FATHMM_pred': {
        'T': 0,
        'D': 1,
    }
}
clinvar_map = {
    'clinvar_clnsig': {
        'Benign': 11,
        'Likely_benign': 12,
        'Likely_pathogenic': 13,
        'Pathogenic': 14,
        'drug_response': 15,
        'histocompatibility': 16,
        'Uncertain_significance': 17,
        'Conflicting_interpretations_of_pathogenicity': 18
    }
}

df = df.replace({'.': -1})
df = df.replace(mapping)
df = df.replace(clinvar_map)
df = df.replace({'\D': -1}, regex=True)

drop_cols = ['clinvar_clnsig']
# df = df.drop(drop_cols, axis=1)

features = df.columns.tolist()

train, test = train_test_split(df, shuffle=False)
print(test.head)

<bound method NDFrame.head of        SIFT_pred  LRT_pred  MutationTaster_pred  FATHMM_pred  clinvar_clnsig
12702         -1         1                    1           -1              13
12703         -1         1                    1           -1              17
12704         -1         1                    1           -1              17
12705          1         1                    1            1              17
12706         -1         1                    1           -1              17
...          ...       ...                  ...          ...             ...
16931          1         2                    0            0              17
16932          0         0                    0            0              17
16933          0         0                    0            0              11
16934          0        -1                    0            0              17
16935         -1         0                    3           -1              11

[4234 rows x 5 columns]>


In [47]:
class DataLoaderdbNSFP(Dataset):
    def __init__(self, data):
        self.data = torch.FloatTensor(data.values.astype('float'))
        
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
            target = self.data[index][-1]
            data_val = self.data[index] [:-1]
            return data_val,target
train_dataset = DataLoaderdbNSFP(train)
valid_dataset = DataLoaderdbNSFP(test)

In [51]:
train_loader = DataLoader(train_dataset, batch_size=1000, shuffle=True)
print(train_loader)